### Thinking1：XGBoost与GBDT的区别是什么？

XGBoost 是对 GBDT 的优化，具体来说XGBoost 的特点可以分为以下几点：
1. XGBoost 将树模型的复杂度加入到正则化项中，从而避免过拟合，泛化性能好
2. 损失函数是用泰勒展开式展开的，用到了一阶导数和二阶导数，可以加快优化速度
3. 在寻找最佳分割点时，采用近似贪心算法，用来加速计算
4. 不仅支持CART作为基分类器，还支持线性分类器，在使用线性分类器的时候可以使用L1、L2正则化
5. 支持并行计算，XGBoost 的并行是基于特征计算的并行，将特征列排序后以 block 的形式存储在内存中，在后面的迭代中重复使用这个结构。在进行节点分裂的时，计算每个特征的增益，选择增益最大的特征作为分割节点，各个特征的增益可以使用多线程并行。
6. 优点：速度快、效果好、能处理大规模数据、支持自定义损失函数等
7. 缺点：算法参数过多、调参复杂，不适合处理超高维特征数据

### Thinking2：举一个你之前做过的预测例子（用的什么模型，解决什么问题，比如我用LR模型，对员工离职进行了预测，效果如何... 请分享到课程微信群中）

- 做过 Titanic 生存率的预测，采用了决策树的模型，在测试集上准确率上达到了 0.985，但是在kaggle 上面提交只有0.83.取得了 top2% 的成绩
- 对原始的数据集进行数据清洗，填充了一些缺失值，去除了几条重复数据，对部分数据集未提供的信息，去Google查找关于titanic相关的资料来补充数据集
- 对原始数据进行了特征工程，按照相关的特征构造出了26 个特征
- 该项目中数据量比较少，所以该项目没有采用深度学习的的方法，后续我会尝试几种看看效果

### Thinking3：请你思考，在你的工作中，需要构建哪些特征（比如用户画像，item特征...），这些特征都包括哪些维度（鼓励分享到微信群中，进行交流）


目前没有参加工作，在读智能制造（面向航空航天）相关的硕士生，比如对于生产过程中的故障诊断，对于一个六轴伺服机械臂上推广分析模型，在分析和采集大量机械臂数据后，比如：手臂某时刻的自由度、速度、加速度、牛顿力等等，对于不同种类和运行工况的机械臂进行聚类分析，形成一个个机械臂的‘虚拟社区’，通过集群搭建方法对社区机械臂进行数据分析，通过比较每一个机械臂与集群的差异性来判断异常程度，并对集群内所有机械臂的健康状态进行排序。

### Action1：男女声音识别


## 方法1：SVM

In [17]:
import pandas as pd
import numpy as np

# 数据加载
df = pd.read_csv('./voice/voice.csv')
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [18]:
df.tail()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
3163,0.131884,0.084734,0.153707,0.049285,0.201144,0.151859,1.762129,6.630383,0.962934,0.763182,...,0.131884,0.182790,0.083770,0.262295,0.832899,0.007812,4.210938,4.203125,0.161929,female
3164,0.116221,0.089221,0.076758,0.042718,0.204911,0.162193,0.693730,2.503954,0.960716,0.709570,...,0.116221,0.188980,0.034409,0.275862,0.909856,0.039062,3.679688,3.640625,0.277897,female
3165,0.142056,0.095798,0.183731,0.033424,0.224360,0.190936,1.876502,6.604509,0.946854,0.654196,...,0.142056,0.209918,0.039506,0.275862,0.494271,0.007812,2.937500,2.929688,0.194759,female
3166,0.143659,0.090628,0.184976,0.043508,0.219943,0.176435,1.591065,5.388298,0.950436,0.675470,...,0.143659,0.172375,0.034483,0.250000,0.791360,0.007812,3.593750,3.585938,0.311002,female
3167,0.165509,0.092884,0.183044,0.070072,0.250827,0.180756,1.705029,5.769115,0.938829,0.601529,...,0.165509,0.185607,0.062257,0.271186,0.227022,0.007812,0.554688,0.546875,0.350000,female


In [19]:
# corr 相关系数的函数
df.corr

<bound method DataFrame.corr of       meanfreq        sd    median       Q25       Q75       IQR       skew  \
0     0.059781  0.064241  0.032027  0.015071  0.090193  0.075122  12.863462   
1     0.066009  0.067310  0.040229  0.019414  0.092666  0.073252  22.423285   
2     0.077316  0.083829  0.036718  0.008701  0.131908  0.123207  30.757155   
3     0.151228  0.072111  0.158011  0.096582  0.207955  0.111374   1.232831   
4     0.135120  0.079146  0.124656  0.078720  0.206045  0.127325   1.101174   
...        ...       ...       ...       ...       ...       ...        ...   
3163  0.131884  0.084734  0.153707  0.049285  0.201144  0.151859   1.762129   
3164  0.116221  0.089221  0.076758  0.042718  0.204911  0.162193   0.693730   
3165  0.142056  0.095798  0.183731  0.033424  0.224360  0.190936   1.876502   
3166  0.143659  0.090628  0.184976  0.043508  0.219943  0.176435   1.591065   
3167  0.165509  0.092884  0.183044  0.070072  0.250827  0.180756   1.705029   

             kurt  

In [20]:
# isnull()判断函数缺失
df.isnull().sum()

meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64

In [21]:
df.shape

(3168, 21)

In [22]:
print('样本的个数：{}'.format(df.shape[0]))
print('男性样本个数：{}'.format(df[df.label == 'male'].shape[0]))
print('女性样本个数：{}'.format(df[df.label == 'female'].shape[0]))

样本的个数：3168
男性样本个数：1584
女性样本个数：1584


In [23]:
# 分割特征和 Target
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
print(X)
print(y)

      meanfreq        sd    median       Q25       Q75       IQR       skew  \
0     0.059781  0.064241  0.032027  0.015071  0.090193  0.075122  12.863462   
1     0.066009  0.067310  0.040229  0.019414  0.092666  0.073252  22.423285   
2     0.077316  0.083829  0.036718  0.008701  0.131908  0.123207  30.757155   
3     0.151228  0.072111  0.158011  0.096582  0.207955  0.111374   1.232831   
4     0.135120  0.079146  0.124656  0.078720  0.206045  0.127325   1.101174   
...        ...       ...       ...       ...       ...       ...        ...   
3163  0.131884  0.084734  0.153707  0.049285  0.201144  0.151859   1.762129   
3164  0.116221  0.089221  0.076758  0.042718  0.204911  0.162193   0.693730   
3165  0.142056  0.095798  0.183731  0.033424  0.224360  0.190936   1.876502   
3166  0.143659  0.090628  0.184976  0.043508  0.219943  0.176435   1.591065   
3167  0.165509  0.092884  0.183044  0.070072  0.250827  0.180756   1.705029   

             kurt    sp.ent       sfm      mode  ce

In [24]:
# 使用标签编码
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
y = gender_encoder.fit_transform(y)
y

array([1, 1, 1, ..., 0, 0, 0])

In [25]:
# 数据规范化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[-4.04924806,  0.4273553 , -4.22490077, ..., -1.43142165,
        -1.41913712, -1.45477229],
       [-3.84105325,  0.6116695 , -3.99929342, ..., -1.41810716,
        -1.4058184 , -1.01410294],
       [-3.46306647,  1.60384791, -4.09585052, ..., -1.42920257,
        -1.41691733, -1.06534356],
       ...,
       [-1.29877326,  2.32272355, -0.05197279, ..., -0.5992661 ,
        -0.58671739,  0.17588664],
       [-1.2452018 ,  2.012196  , -0.01772849, ..., -0.41286326,
        -0.40025537,  1.14916112],
       [-0.51474626,  2.14765111, -0.07087873, ..., -1.27608595,
        -1.2637521 ,  1.47567886]])

In [26]:
# 切分训练集和测试集
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [27]:
# 调用模型
from sklearn.svm import SVC
svc = SVC()

# 训练
svc.fit(X_train,y_train)
# 测试
y_pred = svc.predict(X_test)

In [28]:
print('SVM 预测结果',y_pred)
from sklearn import metrics
# y_test， y_pred 顺序在准确率计算中，顺序颠倒之后结果不变
print('SVM 准确率：',metrics.accuracy_score(y_test,y_pred))

SVM 预测结果 [1 1 1 1 0 1 1 1 0 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 1
 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 0 0
 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0
 1 0 0 1 1 0 0 0 1 1 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 0 0 1 0
 0 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0
 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0
 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1
 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 1 0 0 0 1
 1 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 1 0 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 1 0 0 1
 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0 0 1 1 0 1 1 1 1 0 0
 0 1 0 0 1 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0
 0 0 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0
 0 1 1 0 0 1 1 1

### 方法2：使用 XGBoost

In [29]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [41]:
param = {
    'boosting_type':'gbdt',
    'objective':'binary:logistic',  # 任务目标
    'eval_metric':'auc',  # 评估指标
    'eta':0.01,  # 学习率
    'max_depth':20,  # 树最大深度
    'colsample_bytree':0.8,  # 设置在每次迭代中使用特征的比例
    'subsample':0.9,  # 样本采样比例
    'subsample+freq':8,  # bagging 的次数
    'alpha':0.6, # L1正则化
    'lambda':0 # L2 正则化
    
}

In [42]:
train_data = xgb.DMatrix(X_train,label = y_train)
test_data = xgb.DMatrix(X_test,label = y_test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (test_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
y_pred = model.predict(test_data)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
print('XGBoost 预测结果',y_pred)
print('XGBoost 预测准确率',metrics.accuracy_score(y_pred,y_test))

[0]	train-auc:0.992324	valid-auc:0.993001
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 200 rounds.
[25]	train-auc:0.999197	valid-auc:0.997062
[50]	train-auc:0.999327	valid-auc:0.997271
[75]	train-auc:0.999369	valid-auc:0.997291
[100]	train-auc:0.999578	valid-auc:0.997376
[125]	train-auc:0.999683	valid-auc:0.997406
[150]	train-auc:0.999761	valid-auc:0.997396
[175]	train-auc:0.999802	valid-auc:0.998474
[200]	train-auc:0.99983	valid-auc:0.998454
[225]	train-auc:0.999864	valid-auc:0.998533
[250]	train-auc:0.99989	valid-auc:0.998613
[275]	train-auc:0.999912	valid-auc:0.998633
[300]	train-auc:0.99993	valid-auc:0.998613
[325]	train-auc:0.99994	valid-auc:0.998583
[350]	train-auc:0.999958	valid-auc:0.998563
[375]	train-auc:0.999969	valid-auc:0.998553
[400]	train-auc:0.999971	valid-auc:0.998573
[425]	train-auc:0.999982	valid-auc:0.998523
[450]	train-auc:0.999989	valid-auc:0.998553
Stopping. Best iteration:
[26